In [182]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing
import datetime as dt
from sklearn import model_selection
from sklearn import 
# Load dataset 
train = pd.read_csv('asset/train.csv')
test = pd.read_csv('asset/test.csv')
sample = pd.read_csv('asset/sampleSubmission.csv')
weather = pd.read_csv('asset/weather.csv')

# Get labels
labels = train.WnvPresent.values

# Creating dummy variables for the weather data
weather.CodeSum = weather.CodeSum.apply(str.split)
for (i,list_) in enumerate(weather.CodeSum):
    for item in list_:
        if item not in weather.columns:
            weather[item] = 0
        weather.set_value(col=item,index=i,value=1)


# Not using codesum anymore
weather = weather.drop('CodeSum', axis=1)
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)
weather.Date = pd.to_datetime(weather.Date)

In [183]:
# for c in weather.columns:
#     print(c)
#     print (weather[c].value_counts())

In [184]:
# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [185]:
# Functions to extract month and day from dataset
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day

# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

In [186]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)

In [187]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')

In [188]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [189]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [190]:
train = train.drop('Date', axis=1)
test = test.drop('Date', axis=1)

In [191]:
train.convert_objects(convert_numeric=True)
test.convert_objects(convert_numeric=True)

/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
/home/roland/anaconda3/envs/ga-immersive/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,year,month,day,...,TS_y,DZ_y,BCFG_y,FG+_y,MIFG_y,FG_y,SQ_y,SN_y,VCFG_y,GR_y
0,2,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
1,3,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
2,1,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
3,4,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
4,6,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
5,5,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
6,7,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
7,0,41,36,1,41.954690,-87.800991,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
8,2,62,30,8,41.994991,-87.769279,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0
9,3,62,30,8,41.994991,-87.769279,9,2008,6,11,...,0,0,0,0,0,0,0,0,0,0


In [229]:
n_estimators = [10, 100, 1000]
criterion = ['gini', 'entropy']
max_depth = [4, 10, 30,None]
max_features = ['auto', 'log2', 2, 4, 6]
clf = ensemble.RandomForestClassifier(min_samples_split=5, class_weight='balanced')

In [ ]:
# Random Forest Classifier 
grid = model_selection.GridSearchCV(estimator=clf, scoring='roc_auc', param_grid=dict(n_estimators=n_estimators, criterion=criterion,
                                                    max_depth=max_depth, max_features=max_features))

grid.fit(train,labels)


In [ ]:
grid.best_estimator_.fit(train, labels)

# create predictions and submission file
predictions = grid.best_estimator_.predict_proba(test)[:,1]
sample['WnvPresent'] = predictions
sample.to_csv('beat_the_benchmark.csv', index=False)

In [ ]:
grid.best_score_

In [ ]:
sum(labels)/float(len(labels))

In [ ]:
np.mean(model_selection.cross_val_score(grid.best_estimator_, X=train, y=labels, scoring='accuracy', n_jobs=7))

In [ ]:
model = grid.best_estimator_
